In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                image                   id  \
1960                  ../data/Train/CameraRGB/846.png                  846   
420                   ../data/Train/CameraRGB/339.png                  339   
1787  ../data/Train/CameraRGB/episode_0001_000274.png  episode_0001_000274   
1087  ../data/Train/CameraRGB/episode_0001_000289.png  episode_0001_000289   
2499                  ../data/Train/CameraRGB/672.png                  672   

                                                label  
1960                  ../data/Train/CameraSeg/846.png  
420                   ../data/Train/CameraSeg/339.png  
1787  ../data/Train/CameraSeg/episode_0001_000274.png  
1087  ../data/Train/CameraSeg/episode_0001_000289.png  
2499                  ../data/Train/CameraSeg/672.png  


In [5]:
from models.fcn8 import model_fcn8

model = model_fcn8(3, image_shape=(480, 480, 3), keep_prob=0.7)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 480, 480, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 480, 480, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 240, 240, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [6]:
from gen.generators import train_and_lab_gen_func

BATCH_SIZE = 16

train_gen = train_and_lab_gen_func(train_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE)
valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE)
test_gen = train_and_lab_gen_func(test_df, image_size=(600, 800),  target_size=(480, 480),  batch_size = BATCH_SIZE)

In [7]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model, 
                   train_gen, 
                   valid_gen, 
                   steps_per_epoch=m//BATCH_SIZE, 
                   validation_steps=valid_df.shape[0]//BATCH_SIZE,
                   output_path='../saved_models/fcn8_extended', 
                   epochs=100)

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/keras/engine/training.py:2095: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Reinserting dataframe: 500 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 500 images
## Ignore next

156/156 [==============================] - 202s 1s/step - loss: 0.0297 - acc: 0.9893 - val_loss: 0.0622 - val_acc: 0.9786

Epoch 00061: val_loss improved from 0.06261 to 0.06225, saving model to ../saved_models/fcn8_extended/model.hdf5
Epoch 62/100
156/156 [==============================] - 202s 1s/step - loss: 0.0296 - acc: 0.9892 - val_loss: 0.0597 - val_acc: 0.9792

Epoch 00062: val_loss improved from 0.06225 to 0.05973, saving model to ../saved_models/fcn8_extended/model.hdf5
Epoch 63/100
156/156 [==============================] - 202s 1s/step - loss: 0.0289 - acc: 0.9893 - val_loss: 0.0582 - val_acc: 0.9796

Epoch 00063: val_loss improved from 0.05973 to 0.05818, saving model to ../saved_models/fcn8_extended/model.hdf5
Epoch 64/100
156/156 [==============================] - 205s 1s/step - loss: 0.0285 - acc: 0.9893 - val_loss: 0.0595 - val_acc: 0.9796

Epoch 00064: val_loss did not improve
Epoch 65/100
156/156 [==============================] - 204s 1s/step - loss: 0.0278 - acc: 0

In [10]:
print(history)
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

None
